In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import numpy as np
import random

torch.manual_seed(1)

### 1. Read and process the data.

In [2]:
path = './data/'

In [67]:
def get_data(name):
    f = open(path + name + '.txt', 'r')
    lines = f.readlines()
    
    result = []
    
    sentence = []
    tag_sentence = []
    
    for line in lines[2:]:
        if line == '\n':           
            
            if len(sentence) > 0:
                result.append((sentence, tag_sentence))
            
                sentence = []
                tag_sentence = []
            
            continue
            
        words = line.strip().split()
        
        if words[0] == '-DOCSTART-':
            continue
        
        sentence.append(words[0])
        tag_sentence.append(words[3])
            
        
    return result

In [68]:
train = get_data('train')
dev = get_data('dev')
test = get_data('test')

dataset = train + dev + test

### 2. Implement 3 strategies for loading the embeddings

In [5]:
embeddings_name = 'glove.6B.100d.txt'

def loadGloveModel(file):
    print("Loading Glove Model")
    f = open(file,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [6]:
gmodel = loadGloveModel(path + embeddings_name)

Loading Glove Model
Done. 400000  words loaded!


In [7]:
EMBEDDING_DIM = 100


In [8]:
vocab = {}
tag_vocab = {}
for sent, tags in dataset:
    for word in sent:
        if word not in vocab:
            vocab[word] = len(vocab)
    for tag in tags:
        if tag not in tag_vocab:
            tag_vocab[tag] = len(tag_vocab)

vocab['pad'] = len(vocab)
            
vocab_size = len(vocab)

In [9]:
tag_vocab

{'B-LOC': 5,
 'B-MISC': 2,
 'B-ORG': 0,
 'B-PER': 3,
 'I-LOC': 8,
 'I-MISC': 7,
 'I-ORG': 6,
 'I-PER': 4,
 'O': 1}

**2.a.** Load the embeddings for original capitalization of words. If embedding for this word doesn’t exists, associate it with UNKNOWN embedding

In [10]:
unk = gmodel['unk']

In [11]:
gmodel_strategy_1 = np.zeros((vocab_size, EMBEDDING_DIM))

for word, index in vocab.items():
    if word in gmodel:
        gmodel_strategy_1[index,:] = gmodel[word]
    else:
        gmodel_strategy_1[index,:] = unk

**2.b.** load the embeddings for lowercased capitalization of words. If embedding for this lowercased word doesn’t exists, associate it with UNKNOWN embedding

In [12]:
gmodel_strategy_2 = np.zeros((vocab_size, EMBEDDING_DIM))

for word, index in vocab.items():
    word_lower = word.lower()
    if word_lower in gmodel:
        gmodel_strategy_2[index,:] = gmodel[word_lower]
    else:
        gmodel_strategy_2[index,:] = unk

**2.c.** load the embeddings for original capitalization of words. If embedding for this word doesn't exists, try to find the embedding for lowercased version and associate it to the word with original capitalization. Otherwise, associate it with UNKNOWN embedding

In [13]:
gmodel_strategy_3 = np.zeros((vocab_size, EMBEDDING_DIM))

for word, index in vocab.items():
    word_lower = word.lower()
    if word in gmodel:
        gmodel_strategy_3[index,:] = gmodel[word]
    elif word_lower in gmodel:
        gmodel_strategy_3[index,:] = gmodel[word_lower]
    else:
        gmodel_strategy_3[index,:] = unk

### 3. Implement training on batches

In [14]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


def prepare_batch(batch_sentences, vocab):
    batch_max_len = max([len(s[0]) for s in batch_sentences])

    #prepare a numpy array with the data, initializing the data with 'PAD' 
    #and all labels with -1; initializing labels to -1 differentiates tokens 
    #with tags from 'PAD' tokens
    batch_data = vocab['pad']*np.ones((len(batch_sentences), batch_max_len))
    batch_labels = -1*np.ones((len(batch_sentences), batch_max_len))

    #copy the data to the numpy array
    for j in range(len(batch_sentences)):
        cur_len = len(batch_sentences[j][0])
        batch_data[j][:cur_len] = prepare_sequence(batch_sentences[j][0], vocab)
        batch_labels[j][:cur_len] = prepare_sequence(batch_sentences[j][1], tag_vocab)

    #since all data are indices, we convert them to torch LongTensors
    batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels)

    return batch_data, batch_labels

    
# get_batch(train[:10], vocab)

In [24]:
class LSTM_NER(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, embedding_matrix, tagset_size):
        super(LSTM_NER, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(embedding_matrix.shape[0], embedding_dim).\
            from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float), freeze=True)            
        self.word_embeddings.weight.requires_grad = False

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, dropout=0.2, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.fc = nn.Linear(2*hidden_dim, tagset_size)
        

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        
        lstm_out, _ = self.lstm(embeds)
        
        lstm_out = lstm_out.view(-1, lstm_out.shape[2])
        
        fc_out = self.fc(lstm_out)
        
        tag_scores = F.log_softmax(fc_out, dim=1)
        return tag_scores

In [70]:
HIDDEN_DIM = 50

BATCH_SIZE = 64

model = LSTM_NER(EMBEDDING_DIM, HIDDEN_DIM, gmodel_strategy_3, len(tag_vocab))
# loss_function = nn.NLLLoss()

def loss_function(outputs, labels):
    #reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)  

    #mask out 'PAD' tokens
    mask = (labels >= 0).float()

    #the number of tokens is the sum of elements in mask
    num_tokens = int(torch.sum(mask).data.item())

    #pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels]*mask

    #cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs)/num_tokens


optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
#     inputs = prepare_sequence(train[0][0], vocab)
    inputs, labels = prepare_batch(train[:BATCH_SIZE], vocab)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-2.1571, -2.1610, -2.1721,  ..., -2.3158, -2.2886, -2.1345],
        [-2.1914, -2.1702, -2.1949,  ..., -2.3141, -2.2587, -2.1367],
        [-2.1551, -2.1499, -2.1952,  ..., -2.3052, -2.2815, -2.1415],
        ...,
        [-2.1512, -2.1951, -2.1636,  ..., -2.2718, -2.2741, -2.1767],
        [-2.1382, -2.1923, -2.1775,  ..., -2.2673, -2.2758, -2.1879],
        [-2.1505, -2.1960, -2.1838,  ..., -2.2717, -2.2569, -2.1819]])


In [40]:
def get_batch(b_size, data):
    batch_sentences = random.sample(data, b_size)
    
    return prepare_batch(batch_sentences, vocab)


def get_accuracy(prediction, target):
    index_global = 0
    counter = 0
    
    classes_tp = np.zeros(len(tag_vocab))
    classes_fp = np.zeros(len(tag_vocab))
    classes_fn = np.zeros(len(tag_vocab))
    
    for row in target:
        for col in row:        
            if col < 0:
                continue

            result = col.item()
            
            if result == 1:
                continue

            max_val, pred = prediction[ index_global ].max(0)

            if pred == result:
                counter += 1
                
                
            if pred == result:
                classes_tp[result] += 1
            else:
                classes_fp[pred] += 1
                classes_fn[result] += 1
            

            index_global += 1

#     print(classes_tp)
#     print(classes_fp)
#     print(classes_fn)

    tp = np.sum(classes_tp)
    fp = np.sum(classes_fp)
    fn = np.sum(classes_fn)

    return tp / (tp + fp), tp / (tp + fn)


def validate_model(model, dataset, vocab, tag_vocab):
    with torch.no_grad():
    
        size = len(dev)

        sentence_in, targets = prepare_batch(dataset, vocab)

        prediction = model(sentence_in)
        
        pr, rc = get_accuracy(prediction, targets)
        loss = loss_function(prediction, targets)

        return pr, rc, loss.item()


# np.round(validate_model(model, test, vocab, tag_vocab), 6)

In [38]:
steps = len(train) // BATCH_SIZE + 1

random.seed(42)

from tqdm import tqdm_notebook

train_sorted = sorted(train, key=lambda item: (len(item[0])))

for epoch in range(5):  # again, normally you would NOT do 300 epochs, it is toy data
    loss_sum = 0
    print("Epoch: {}".format(epoch + 1))
    for step in tqdm_notebook(range(steps)):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        
        # sentence_in, targets = get_batch(BATCH_SIZE, train)        
        sentence_in, targets = prepare_batch(train_sorted[step * BATCH_SIZE: (step+1) * BATCH_SIZE], vocab)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
        loss_sum += loss
        
    avg_epoch_loss = np.round((loss_sum / steps).item(), 6)
    dev_epoch_pr, dev_epoch_rc, dev_epoch_loss = np.round(validate_model(model, dev, vocab, tag_vocab), 6)
    
    print("Train loss:     {}\nValidate loss:  {}\nValidate accuracy: {}".\
    format(avg_epoch_loss, dev_epoch_loss, dev_epoch_pr))

Epoch: 1


HBox(children=(IntProgress(value=0, max=235), HTML(value='')))

Train loss:     0.850249
Validate loss:  0.693659
Validate accuracy: 0.0


In [ ]:
validate_model(model, test, vocab, tag_vocab)